This file is used to define important classes and functions which are used in the main analysis

# Phase and signal

reset_F (generic function with 1 method)

iterate_process (generic function with 1 method)

# Various functions

In [1]:
#build coupling from Gaussian
function build_F_from_2D_gaussian(l_v_mu, l_mat_sigma, l_amp, v_domain)
    flat_v_res = zeros( size(v_domain,1)*size(v_domain,1))
    mat_domain =  [ [theta, phi] for theta in v_domain, phi in v_domain ] 
    flat_mat_domain = reshape(mat_domain, size(mat_domain,1)*size(mat_domain,2) )
    flat_mat_domain = hcat(flat_mat_domain...) #flatten over last dimension
    for (v_mu, mat_sigma, amp) in zip(l_v_mu, l_mat_sigma, l_amp)
        flat_v_res += amp * pdf( MvNormal(  v_mu, mat_sigma), flat_mat_domain)
        flat_v_res += amp * pdf( MvNormal( [v_mu[1]+2*π, v_mu[2]], mat_sigma), flat_mat_domain)
        flat_v_res += amp * pdf( MvNormal( [v_mu[1], v_mu[2]+2*π], mat_sigma), flat_mat_domain)
        flat_v_res += amp * pdf( MvNormal( [v_mu[1]+2*π, v_mu[2]+2*π], mat_sigma), flat_mat_domain)
    end
    return reshape(flat_v_res, (size(mat_domain,1),size(mat_domain,2)) )
end


#plot coupling function
function plot_F(F; title_fig = " ", cmap = "bwr", vmin = -0.3, vmax = 0.3, interpolation = "bessel", save = false)
    figure()
    imshow(permutedims(F, (2,1)), origin = "lower", interpolation = interpolation , cmap = cmap, vmin = vmin, vmax = vmax)
    colorbar()
    xlabel("θ")
    ylabel("ϕ")
    title(title_fig)
    if save
        savefig("Figures/" *title_fig*".png")
    end
    #show()
end

function angular_subtract(x,y)
    x, y = real.(x), real.(y)
    z = (x .- y).%(2*π)
    for i in 1:size(z,1)
        if z[i] > π
            z[i] -= 2*π
        elseif z[i] < -π
            z[i] += 2*π
        end
    end
    return z
end
               


angular_subtract (generic function with 1 method)

# Sigma points

In [1]:
#comptue weights of the sigma points
function compute_weights(dim_x, α, β, κ)

    lambda_ = α^2 * (dim_x + κ) - dim_x

    c = .5 / (dim_x + lambda_)
    Wc = fill(c, 2*dim_x + 1)
    Wm = fill(c, 2*dim_x + 1)
    Wc[1] = lambda_ / (dim_x + lambda_) + (1 - α^2 + β)
    Wm[1] = lambda_ / (dim_x + lambda_)
    return (Wc, Wm)
end

#mean of a weighted angular variable #TO BE REDEFINED WHEN THE SYSTEM CHANGES
function sigma_angular_mean(sigmas, Wm)
    x = zeros(size(sigmas,2))
    sum_sin_1 = 0.
    sum_cos_1 = 0.
    sum_sin_2 = 0.
    sum_cos_2 = 0.
    for i in 1:size(sigmas,1)
        s = sigmas[i,:]
        sum_sin_1 += sin(s[1])*Wm[i]
        sum_cos_1 += cos(s[1])*Wm[i]
        sum_sin_2 += sin(s[2])*Wm[i]
        sum_cos_2 += cos(s[2])*Wm[i]
    end
    x[1] = mod2pi(atan(sum_sin_1, sum_cos_1))
    x[2] = mod2pi(atan(sum_sin_2, sum_cos_2))
    return x
end
        

sigma_angular_mean (generic function with 1 method)

# URTS smoother

In [2]:
mutable struct URTS
    x::Vector{Float64} #state estimate vector
    P::Matrix{Float64} #covariance estimate matrix
    R::Matrix{Float64} #measurement noise matrix
    Q::Matrix{Float64} #process noise matrix
    K::Matrix{Float64} #Kalman gain
    y::Vector{Float64} #innovation residual
    z::Vector{Float64} #measurment
    S::Matrix{Float64} # system uncertainty
    SI::Matrix{Float64} # inverse system uncertainty
    
    dim_x::Int32
    dim_z::Int32
    num_sigmas::Int32
    dt::Float64
    fx 
    hx
    
    # weights for the means and covariances.
    Wm::Vector{Float64}
    Wc::Vector{Float64}

    sigmas_f::Matrix{Float64}
    sigmas_h::Matrix{Float64}
    
    α::Float64
    β::Float64
    κ::Float64
    
    zp::Vector{Float64}
    
    ll::Vector{Float64} #log-likelihood vector
end

"""
URTS constructor
"""
function URTS(;dim_x, dim_z, dt, hx, fx, α, β, κ)
    x = zeros(dim_x)
    P = Matrix(I, dim_x, dim_x)
    Q = Matrix(I, dim_x, dim_x)
    R = Matrix(I, dim_x, dim_x)
    dim_x = dim_x
    dim_z = dim_z
    num_sigmas = 2*dim_x + 1

    # weights for the means and covariances.
    Wc, Wm = compute_weights(dim_x, α, β, κ)

    sigmas_f = zeros(num_sigmas, dim_x)
    sigmas_h = zeros(num_sigmas, dim_z)

    K = zeros(dim_x, dim_z)    # Kalman gain
    y = zeros(dim_z)           # residual
    
    z = zeros(dim_z )  # measurement
    S = zeros(dim_z, dim_z)    # system uncertainty
    SI = zeros(dim_z, dim_z)   # inverse system uncertainty
    
    zp = zeros(dim_z)
    ll = Float64[]
    
    return URTS(x, P, R, Q, K, y, z, S, SI, dim_x, dim_z, num_sigmas, dt, fx, hx, Wm, Wc, sigmas_f, sigmas_h, α, β, κ, zp, ll)
end
    
# TO BE CORRECTED FOR ANGULAR VARIABLE
function unscented_transform(urts::URTS, pass = "state")
    if pass == "state"
        
        """
        (kmax, n) = size(urts.sigmas_f)

        x = sigma_angular_mean(urts.sigmas_f, urts.Wm)
        #println("x UT", x)
        
        P = zeros(n, n)
        for k in 1:kmax
            
            y = angular_subtract(urts.sigmas_f[k,:], x)
            P += urts.Wc[k] .* (y.*y')
        end
        #println("P", P)
        P += urts.Q
        
        urts.x = x
        urts.P = P
        """
                
        x = urts.sigmas_f' * urts.Wm
        y = urts.sigmas_f .- x'
        
        
        #println(size(y))
        #println(size(urts.Wc))
        #println(size(urts.Wc .* y))
        
        P = y' * (urts.Wc .* y)
        P += urts.Q

        urts.x = x
        urts.P = P
        
        
    elseif pass == "obs"
    
        x = urts.sigmas_h' * urts.Wm
        y = urts.sigmas_h .- x'
        P = y' * (urts.Wc .* y)
        P += urts.R

        urts.zp = x
        urts.S = P
    end
            
end                            
                            
                            
function predict(urts::URTS; l_arg_f = [])
    sigmas = sigma_points(urts.α, urts.β, urts.κ, urts.x, urts.P)
    
    for i in 1:size(sigmas,1)
        urts.sigmas_f[i,:] = urts.fx(sigmas[i,:], dt, l_arg_f...)
    end
    
    #println(urts.sigmas_f[:,:])
        
    #println("first print predict", urts.x, urts.P, urts.sigmas_f, urts.Wm, urts.Wc, urts.Q)
        
    unscented_transform(urts, "state")
    #println("second print predict", urts.x, urts.P)
end


function update(urts::URTS, z)

    for i in 1:size(urts.sigmas_f,1)
        urts.sigmas_h[i,:] =  urts.hx(urts.sigmas_f[i,:])
    end
    
    #println("first print update", urts.x, urts.P, urts.sigmas_h, z, urts.R)
    
    # mean and covariance of prediction passed through unscented transform
    unscented_transform(urts, "obs")
    log_likelihood_current_obs = loglikelihood(urts, z)
    push!(ll, log_likelihood_current_obs)
        
    urts.SI = inv(urts.S)

    # compute cross variance of the state and the measurements
    Pxz = cross_variance(urts)
        
    #println("second print update", urts.zp, urts.S, Pxz, urts.SI)
        
    urts.K = Pxz * urts.SI        # Kalman gain
    urts.y = z .- urts.zp         # residual

    # update Gaussian state estimate (x, P)
    #println("critical point",urts.K , urts.y, urts.x)
    #urts.x = mod2pi.(urts.x + urts.K * urts.y)
    urts.x = urts.x + urts.K * urts.y
    urts.P = urts.P - urts.K *( urts.S * urts.K')
    #println("third print update",urts.K , urts.y, urts.x, urts.P)
        
    #println("x after update", urts.x)
end
 
 
function cross_variance(urts::URTS)
    Pxz = zeros(size(urts.sigmas_f,2), size(urts.sigmas_h,2))
    N = size(urts.sigmas_f,1)
    for i in 1:N
        dx = urts.sigmas_f[i,:] .- urts.x #angular_subtract(urts.sigmas_f[i,:], urts.x)  #
        dz = urts.sigmas_h[i,:] .- urts.zp
        Pxz += urts.Wc[i,:] .* (dx .* dz')
    end
    return Pxz
end
    
    
function urts_smoother(urts::URTS, x_inf, p_inf; l_arg_f = [])

    n, dim_x = size(x_inf)

    
    dts = [urts.dt for i in 1:n]
    Qs = [urts.Q for i in 1:n]

    # smoother gain
    Ks = zeros(n, dim_x, dim_x)

    num_sigmas = urts.num_sigmas

    xs, ps, Pp = copy(x_inf), copy(p_inf), copy(p_inf)
    sigmas_f = zeros(num_sigmas, dim_x)

    for k in n-1:-1:1
        # create sigma points from state estimate, pass through state func
        sigmas = sigma_points(urts.α, urts.β, urts.κ, xs[k,:], ps[k,:,:])
        for i in 1:num_sigmas
            urts.sigmas_f[i,:] = urts.fx(sigmas[i,:], dts[k], l_arg_f...)
            unscented_transform(urts, "state")
        end
        xb, Pb = urts.x, urts.P
                
        # compute cross variance
        Pxb = 0
        for i in 1:num_sigmas
            y = urts.sigmas_f[i,:] .- xb  #angular_subtract(urts.sigmas_f[i,:], xb)
            z = sigmas[i,:] .- x_inf[k,:] #angular_subtract(sigmas[i,:], x_inf[k,:])
            Pxb = Pxb .+ urts.Wc[i,:] .* (z .* y')
        end

        # compute gain
        K = Pxb * inv(Pb)

        # update the smoothed estimates
        xs[k,:]   .+=  K * (xs[k+1,:] .- xb )  #angular_subtract(xs[k+1,:], xb)
        ps[k,:,:] .+=  K * (ps[k+1,:,:] .- Pb) * K'
        Ks[k,:,:] = K
        Pp[k,:,:] = Pb
    end
    return (xs, ps, Ks, Pp)    
end
        

function joint_distribution(urts::URTS, Ps, x, P, K, Pp)
    
    a_jP_x =  Array{Float64}(undef, 0, 2, urts.dim_x) # dim t, dim joint_t, dim_x
    a_jP_p = Array{Float64}(undef, 0, 2*urts.dim_x, 2*urts.dim_x)
     

    for k in 1:size(x,1)-1

            
        P2 = Ps[k,:,:].-(K[k,:,:] * Pp[k,:,:]) * K[k,:,:]'


        a_jP_x = vcat(a_jP_x, reshape([x[k+1,:]'; x[k,:]'], (1,urts.dim_x, urts.dim_x))   )


        new_p = [ [ P[k+1,:,:] P[k+1,:,:]*K[k,:,:]' ] ;
                [ K[k,:,:] * P[k+1,:,:]  (K[k,:,:] * P[k+1,:,:]) * K[k,:,:]' .+ P2 ] ]
        new_p = reshape( new_p, (1,size(new_p)...))
        a_jP_p = vcat(a_jP_p, new_p)
    end
    return a_jP_x, a_jP_p
end
    
function loglikelihood(urts::URTS, z)
    #to be called at the end of update step
    return -0.5 * ( log(2*pi*abs.(urts.S)) + (z-urts.zp)' * inv(urts.S) * (z-urts.zp) )
end
    
function total_ll(urts::URTS)
    return sum(urts.ll)
end
         

total_ll (generic function with 1 method)

## Coupling

Discretized version

In [1]:
using TensorOperations
function compute_coupling_discretization(l_x, l_p, l_jP_x, l_jP_P, resolution, lambd, theta_f = Theta.f, phi_f = Phi.f, dt = dt )
    
    #println(typeof(l_jP_x))
    dim_x = size(l_x[1][1,:],1)
    
    F_theta_bad = zeros(Float64, resolution, resolution)
    F_phi_bad =   zeros(Float64, resolution, resolution)
    F_count_bad = zeros(Float64, resolution, resolution)
    
    F_theta = zeros(Float64, resolution, resolution)
    F_phi =   zeros(Float64, resolution, resolution)

    #create 2d domain with every couple of phase
    domain = range(0, stop=2*π, length=resolution+1)[1:end-1]
    domain_large = range(-2*π, stop=4*π, length=3*resolution+1)[1:end-1]
    domain_2d_large =  zeros(Float64, 3*resolution, 3*resolution, dim_x)
    for (idx_theta, theta) in enumerate(domain_large)
        for (idx_phi, phi) in enumerate(domain_large)
        domain_2d_large[idx_theta,idx_phi,:] = [theta, phi]
        end
    end

    
    
    flat_domain_2d_large = reshape(domain_2d_large, size(domain_2d_large,1)*size(domain_2d_large,2),dim_x )
    
    #transition matrices
    Mat_tr_theta = [  mod2pi(theta+theta_f*dt) for theta in domain, phi in domain ]
    Mat_tr_phi   = [  mod2pi(phi+phi_f*dt) for theta in domain, phi in domain ]
        
    #create empty array for estimating angles
    Mat_theta_end = zeros(ComplexF64, resolution,resolution)
    Mat_phi_end = zeros(ComplexF64, resolution,resolution)
    
    Mat_theta_end_real = zeros(Float64, resolution,resolution)
    Mat_phi_end_real = zeros(Float64, resolution,resolution)
    
    Mat_norm = zeros(Float64, resolution,resolution)

    Mat_theta_end_large = zeros(ComplexF64, 3*resolution,3*resolution)
    Mat_phi_end_large =   zeros(ComplexF64, 3*resolution,3*resolution)
    Mat_norm_large = zeros(Float64, 3*resolution,3*resolution)
    Mat_cond_mu_large = zeros(Float64, 3*resolution,3*resolution) #TO DELETE ?

    
    for (x, p, jP_x, jP_P) in zip(l_x, l_p, l_jP_x, l_jP_P)
        for t in 1:size(x,1)-1
                    
            
            #compute the conditional normal distribution of theta_tdt and phi_tdt given theta_t and phi_t            
            A = jP_P[t,1:2,3:4] * inv(jP_P[t,3:4,3:4])
            B = domain_2d_large[:,:,:] .- reshape(mod2pi.(jP_x[t,2,:]),1,1,size(jP_x[t,2,:],1))
            
            @tensor begin
            C[1,2,3] := A[3,4]*B[1,2,4]
            end
            
            
            Mat_cond_mu_large = reshape(mod2pi.(jP_x[t,1,:]),1,1, size(mod2pi.(jP_x[t,1,:]),1),1) .+ C
            
            Mat_cond_sigma = jP_P[t,1:2,1:2]- (jP_P[t,1:2,3:4] * inv(jP_P[t,3:4,3:4])) * jP_P[t,3:4,1:2]

            #compute probability of being at a given state using wrapped normal law
            
            #ugly rouding but can't do better with Julia 1.0  
            flat_p_x_large = pdf( MvNormal( mod2pi.(x[t,:]),  round.(p[t,:,:] .* 10^4 ) / 10^4 ) , flat_domain_2d_large' )  
            Mat_p_x_large = reshape(flat_p_x_large, size(domain_2d_large,1),size(domain_2d_large,2) ) 
            Mat_p_x_large = Mat_p_x_large ./ sum(Mat_p_x_large)

            Mat_theta_end_large .+= exp.(1im .* Mat_cond_mu_large[:,:,1] .- Mat_cond_sigma[1,1] ./ 2) .* Mat_p_x_large
            Mat_phi_end_large .+= exp.(1im .* Mat_cond_mu_large[:,:,2] .- Mat_cond_sigma[2,2] ./ 2) .* Mat_p_x_large
            Mat_norm_large .+= Mat_p_x_large
        end
        
        #println(Mat_theta_end_large[25,25])
        #println(Mat_norm_large[25,25])
        
        #println(x)
        #println(size(x))
        v = (x[2:end,:] .- x[1:end-1,:]) ./ dt
        for (ph1, ph2,v1,v2) in zip(x[1:size(v,1),1],x[1:size(v,1),2],v[:,1],v[:,2])
            ph1_idx::Int64 = round( mod2pi.(ph1)/(2*π)*resolution)%(resolution)
            ph2_idx::Int64 = round( mod2pi.(ph2)/(2*π)*resolution)%(resolution)
            F_theta_bad[ph1_idx+1, ph2_idx+1] += v1
            F_phi_bad[ph1_idx+1, ph2_idx+1] += v2 
            F_count_bad[ph1_idx+1, ph2_idx+1] += 1
        end
        
    end

    #sum wrapped distribution
    for idx_periodicity_theta in 0:2
        for idx_periodicity_phi in 0:2
            Mat_theta_end .+= Mat_theta_end_large[resolution*idx_periodicity_theta+1:resolution*(idx_periodicity_theta+1),resolution*idx_periodicity_phi+1:resolution*(idx_periodicity_phi+1)]
            Mat_phi_end .+= Mat_phi_end_large[resolution*idx_periodicity_theta+1:resolution*(idx_periodicity_theta+1),resolution*idx_periodicity_phi+1:resolution*(idx_periodicity_phi+1)]
            Mat_norm .+= Mat_norm_large[resolution*idx_periodicity_theta+1:resolution*(idx_periodicity_theta+1),resolution*idx_periodicity_phi+1:resolution*(idx_periodicity_phi+1)]
        end
    end
    
    
    #println(Mat_theta_end[1,:])
    #compute coupling
    Mat_theta_end_real = mod2pi.(angle.(Mat_theta_end ./ Mat_norm))
    Mat_phi_end_real = mod2pi.(angle.(Mat_phi_end ./ Mat_norm))
    #println(Mat_theta_end)

    #diff angle
    Mat_diff_theta = Mat_theta_end_real .- Mat_tr_theta
    Mat_diff_phi = Mat_phi_end_real .- Mat_tr_phi

    #correct for wrong values
    for idx_theta in 1:resolution
        for idx_phi in 1:resolution
            if Mat_diff_theta[idx_theta, idx_phi] < -π
                Mat_diff_theta[idx_theta, idx_phi] += 2*π
            elseif Mat_diff_theta[idx_theta, idx_phi] > π
                Mat_diff_theta[idx_theta, idx_phi] -= 2*π
            end

            if Mat_diff_phi[idx_theta, idx_phi] < -π
                Mat_diff_phi[idx_theta, idx_phi] += 2*π
            elseif Mat_diff_phi[idx_theta, idx_phi]>π
                Mat_diff_phi[idx_theta, idx_phi] -= 2*π
            end
        end
    end
    
    #compute final coupling

    F_theta = Mat_diff_theta ./ dt
    F_phi = Mat_diff_phi ./ dt
    
    #regularize
    F_theta = F_theta .* Mat_norm .* dt
    F_phi = F_phi .* Mat_norm .* dt
    
    F_theta = F_theta ./ (Mat_norm .* dt .+ lambd .+ 10^-15)
    F_phi = F_phi ./ (Mat_norm .* dt .+ lambd .+ 10^-15)
 
    
    F_theta_bad = F_theta_bad ./ F_count_bad
    F_phi_bad = F_phi_bad ./ F_count_bad
    F_theta_bad = F_theta_bad .- Theta.f
    F_phi_bad = F_phi_bad .- Phi.f
    

    #get fourier transform
    FT_theta = fft(F_theta) 
    FT_phi = fft(F_phi)
    
    return F_theta, F_phi, F_theta_bad, F_phi_bad, Mat_norm, FT_theta, FT_phi
    
end

compute_coupling_discretization (generic function with 4 methods)

Fourier version

In [2]:

function f_hat(x, n_harm)
    """
    vect_exp = (1/n_harm^2).*[ exp(1im * ( k*x[1] + l*x[2] )) for k in 0:n_harm-1 for l in 0:n_harm-1 ] 
    f_hat = vcat( [ 1, x... ], vect_exp )
    """
    
    """
    vect_cos = (1/n_harm^2).*[ cos( k*x[1] + l*x[2] ) for k in 0:n_harm-1 for l in 0:n_harm-1 ]
    vect_sin =  (1/n_harm^2).*[ -sin( k*x[1] + l*x[2] ) for k in 0:n_harm-1 for l in 0:n_harm-1 ]
    f_hat = vcat( [ 1, x... ], vect_cos, vect_sin )
    """
    
    """
    vect_cos = (1/n_harm^2).*[ cos( k*x[1] + l*x[2] ) for k in 0:n_harm-1 for l in 0:n_harm-1 ]
    vect_sin =  (1/n_harm^2).*[ -sin( k*x[1] + l*x[2] ) for k in 0:n_harm-1 for l in 0:n_harm-1 ]
    f_hat = vcat( [ 1, x... ], vect_cos )
    """
    
    f_hat = [ 1, x... ]
    return f_hat
end



function compute_coupling_fourier_in_loop(α, β, κ, xt, pt, jP_xt, jP_Pt, Wm, jWm, dim_x, n_harm, sigmas_f_hat, sigmas_f_hat_c)
    #compute PHI (only until T-1, and starting from k=1 instead of k=0)

    #sigmas_phi = sigma_points(α, β, κ, xt, pt)


    #for i in 1:size(sigmas_phi,1)
    #    sigmas_f_hat[i,:] = f_hat(sigmas_phi[i,:], n_harm)
    #end
    #PHI_k = sigmas_f_hat' * (Wc .* sigmas_f_hat)
    
    
    ###correct phase
    mod_jP_xt = mod2pi.(jP_xt)
    for i in 1:2
        if abs(mod_jP_xt[1,i]-mod_jP_xt[2,i])>π
            mod_jP_xt[1,i] =  mod_jP_xt[2,i] +  jP_xt[1,i]-jP_xt[2,i]
        end
    end
    
    println(mod_jP_xt)
    
    #compute C (starting from k=1 instead of k=0)
    #sigmas_c = sigma_points(α, β, κ, hcat(jP_xt...)', jP_Pt)
    sigmas_c = sigma_points(α, β, κ, hcat(mod_jP_xt...)', jP_Pt)
    
    
    
    for i in 1:size(sigmas_c,1)
        sigmas_f_hat_c[i,:] = f_hat(sigmas_c[i,3:4], n_harm)
    end
    
    
    #println(sigmas_f_hat_c)
    #println(jWm)
    
    PHI_k = sigmas_f_hat_c' * (jWm .* sigmas_f_hat_c)
    
    #println(PHI_k)
    
    #C_k = zeros(ComplexF64, dim_x, size(sigmas_f_hat_c,2))
    C_k = zeros(Float64, dim_x, size(sigmas_f_hat_c,2))
    N = size(sigmas_f_hat_c,1)
    for i in 1:N
        dx = sigmas_c[i,1:2]
        fx = sigmas_f_hat_c[i,:]
        C_k += jWm[i] .* (dx .* fx')
    end
    
    return PHI_k, C_k
end


function compute_coupling_fourier_in_loop_discrete(xt, pt, jP_xt, jP_Pt, dim_x, n_harm, flat_domain_2d_large, flat_domain_4d_large, flat_f_hat_prod, flat_f_hat_x_prod )
    println("start")
    ###try to compute the expectation using the old way
    flat_Mat_P = pdf( MvNormal( mod2pi.(xt),  round.(pt .* 10^4 ) / 10^4 ) , flat_domain_2d_large' )
    flat_Mat_P_double = pdf( MvNormal( mod2pi.( reshape(jP_xt',dim_x*2) ),  round.(jP_Pt .* 10^4 ) / 10^4 ) , flat_domain_4d_large' )
    Mat_P_double = reshape(flat_Mat_P_double, size(flat_domain_2d_large,1), size(flat_domain_2d_large,1))
    println("middle")
    @tensor begin
    bi_flat_Mat_P[a,b] := flat_Mat_P[a]*flat_Mat_P[b]
    PHI_k[b,d] := flat_f_hat_prod[a,b,c,d]*bi_flat_Mat_P[a,c]
    C_k[d,b] := flat_f_hat_x_prod[a,b,c,d]*Mat_P_double[c,a]
    end
    println("end")
    return PHI_k, C_k
end




function compute_coupling_fourier(FT_θ, FT_ϕ, n_harm, l_x, l_p, l_jP_x, l_jP_P, α, β, κ, lambd, theta_f = Theta.f, phi_f = Phi.f, dt = dt )
    #println("OK")
    ### TEST BEFORE
    
    #A = [ hcat([theta_f*dt 1 0], reshape(FT_θ', size(FT_θ,1)* size(FT_θ,2) )' .* dt) ; 
    #      hcat([phi_f*dt 0 1], reshape(FT_ϕ', size(FT_ϕ,1)* size(FT_ϕ,2) )' .* dt)  ]
    
    #A = [ hcat([theta_f*dt 1 0], real.(reshape(FT_θ', size(FT_θ,1)* size(FT_θ,2) ))' .* dt, imag.(reshape(FT_θ', size(FT_θ,1)* size(FT_θ,2) ))' .* dt) ; 
    #      hcat([phi_f*dt 0 1], real.(reshape(FT_ϕ', size(FT_ϕ,1)* size(FT_ϕ,2) ))' .* dt, imag.(reshape(FT_ϕ', size(FT_ϕ,1)* size(FT_ϕ,2) ))' .* dt)  ]
    
    #A = [ hcat([theta_f*dt 1 0], real.(reshape(FT_θ', size(FT_θ,1)* size(FT_θ,2) ))' .* dt) ; 
    #      hcat([phi_f*dt 0 1], real.(reshape(FT_ϕ', size(FT_ϕ,1)* size(FT_ϕ,2) ))' .* dt)  ]
    
    A = [ [theta_f*dt 1 0] ; [phi_f*dt 0 1]  ]
    
    
    
    t = [2, 1]
    t_dt = A*f_hat(t, n_harm)
    println("BEFORE ", mod2pi.(real.(t_dt)))
    
    
    dim_x = size(l_x[1][1,:],1)
    dim_f_hat = 1 + dim_x #+ n_harm * n_harm #*2 #remove *2 if work with complex
    
    #PHI = zeros(ComplexF64, dim_f_hat, dim_f_hat ) 
    #C =  zeros(ComplexF64, dim_x, dim_f_hat )
    PHI = zeros(Float64, dim_f_hat, dim_f_hat )
    C =  zeros(Float64, dim_x, dim_f_hat )
    
    num_sigmas = 2*dim_x + 1
    num_sigmas_c = 4*dim_x + 1
    
    #sigmas_f_hat = zeros(ComplexF64, num_sigmas, dim_f_hat)
    #sigmas_f_hat_c = zeros(ComplexF64, num_sigmas_c, dim_f_hat)
    sigmas_f_hat = zeros(Float64, num_sigmas, dim_f_hat)
    sigmas_f_hat_c = zeros(Float64, num_sigmas_c, dim_f_hat)
                    
    #compute sigma weights
    (Wc, Wm) = compute_weights(dim_x, α, β, κ)
    
    #compute sigma weights
    (jWc, jWm) = compute_weights(dim_x*2, α, β, κ)
    
    
    T = 0
    
    """
    ### OLD WAY
    resolution = 20
    domain_large = range(-2*π, stop=4*π, length=3*resolution+1)[1:end-1]
    domain_2d_large =  zeros(Float64, 3*resolution, 3*resolution, dim_x)
    domain_4d_large = zeros(Float64, 3*resolution, 3*resolution, 3*resolution, 3*resolution, dim_x*2)
    f_hat_large = zeros(size(domain_2d_large,1), size(domain_2d_large,2), dim_f_hat)
    for (idx_theta, theta) in enumerate(domain_large)
        for (idx_phi, phi) in enumerate(domain_large)
            domain_2d_large[idx_theta,idx_phi,:] = [theta, phi]
            f_hat_large[idx_theta,idx_phi,:] = f_hat( [theta, phi], n_harm)
            for (idx_theta2, theta2) in enumerate(domain_large)
                for (idx_phi2, phi2) in enumerate(domain_large)
                    domain_4d_large[idx_theta,idx_phi,idx_theta2,idx_phi2,:] = [theta, phi, theta2, phi2]
                end
            end
        end
    end
    
    flat_domain_2d_large = reshape(domain_2d_large, size(domain_2d_large,1)*size(domain_2d_large,2),dim_x )
    flat_domain_4d_large = reshape(domain_4d_large, size(domain_2d_large,1)^2*size(domain_2d_large,2)^2,dim_x*2 )
    flat_f_hat_large = reshape(f_hat_large, size(domain_2d_large,1)*size(domain_2d_large,2),dim_f_hat )
    @tensor begin
    flat_f_hat_prod[1,2,3,4] := flat_f_hat_large[1,2]*flat_f_hat_large[3,4]
    flat_f_hat_x_prod[1,2,3,4] :=  flat_f_hat_large[1,2]*flat_domain_2d_large[3,4]
    end
    """
    

    
    for (x, p, jP_x, jP_P) in zip(l_x, l_p, l_jP_x, l_jP_P)
        for k in 1:size(x,1)-1            
            PHI_k, C_k = compute_coupling_fourier_in_loop(α, β, κ, x[k,:], p[k,:,:], jP_x[k,:,:], jP_P[k,:,:], Wm, jWm, dim_x, n_harm,sigmas_f_hat, sigmas_f_hat_c)
            #PHI_k, C_k = compute_coupling_fourier_in_loop_discrete( x[k,:], p[k,:,:], jP_x[k,:,:], jP_P[k,:,:], dim_x, n_harm, flat_domain_2d_large, flat_domain_4d_large,flat_f_hat_prod, flat_f_hat_x_prod)
            
            PHI = PHI .+ PHI_k
            C = C .+ C_k
            T=+1
        end
    end
    PHI = PHI ./ T
    C = C ./ T

    println("PHI", PHI)
    println("C", C)
    #println(PHI)
    #println(C)
    A = C * inv(PHI)
    #println(inv(PHI))
    
    #FT_theta = reshape(A[1, 1 + dim_x + 1 : end ], n_harm, n_harm)
    #FT_phi = reshape(A[2, 1 + dim_x + 1 : end ], n_harm, n_harm)
    FT_theta = 0#reshape(A[1, 1 + dim_x + 1 : end ], n_harm, n_harm)
    FT_phi = 0 #reshape(A[2, 1 + dim_x + 1 : end ], n_harm, n_harm)
    
    ### TEST
    println(A)
    x = [2., 1.]
    x_dt = A*f_hat(x, n_harm)
    #println(x_dt)
    println("AFTER", mod2pi.(x_dt))
    
    return FT_theta, FT_phi
end


        

compute_coupling_fourier (generic function with 4 methods)